# Anomaly Detection in wood

In [1]:
import glob
import cv2
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import torch
from Model import MyNet
import torch.optim as optim
import torch.nn
from pathlib import Path
torch.manual_seed(17)
np.random.seed(42)

Puisqu'un défaut dans le bois n'est pas fonction de la grandeur on peut augmenter notre dataset en sélectionnant des parties aléatoires (RandomCrop). le choix de 224x224 est arbitraire. 

Dans le jeu de données toutes les images sont orientées dans la même direction (le grain du bois). Ce ne serait surement pas toujours le cas donc il vaut mieux ajouter une rotation dans les images ($\pm 180 \degree$). Cela permet d'éviter loverfitting sur les quelques défauts présents dans le jeu de données. On ne voudrait pas que le modele apprenne à repérer seulement certains types de fissure parce que celles-ci sont dans la bonne orientation

Dans le problème actuel la couleur du bois est peu (pas) importante. On peut donc convertir nos images en noir et blanc

In [2]:
pictures_size = 224
transform = transforms.Compose([
    transforms.RandomCrop(pictures_size), 
    transforms.RandomRotation(180), 
     transforms.Grayscale(),
     transforms.ToTensor(),
     
 ])

dataset = datasets.ImageFolder("data", transform=transform)
dataloader = DataLoader(dataset, batch_size=7, shuffle=True)



Pour résoudre le problème de façon non supervisé j'ai cherché, et trouvé, l'article suivant : https://arxiv.org/pdf/2007.09990.pdf

Ils ont rendu le code disponible sur github : https://github.com/kanezaki/pytorch-unsupervised-segmentation-tip

J'ai donc utilisé leur modèle ainsi que la "costom" loss qui permet de pénaliser en fonction de la distance des pixel ainsi que de la variation entre la couleur (niveau de gris)

On peut ajuster la sensibilité en modition les "stepsize_con" et "stepsize_sim" 

On pourrait aussi utiliser l'option "scribble" en entourant les anomalies (ca devient alors un probleme (semi) supervisé)

In [9]:
nepoch = 100
max_nb_class = 5

visualize = False
scribble =False

stepsize_scr = 1
stepsize_con = 1
stepsize_sim = 3

use_cuda = torch.cuda.is_available()

# MyNet(number_of_channel, maximum_number_of_classes, number_of_convolution + 2)
model = MyNet(1, max_nb_class, 2)

model.cuda() if use_cuda else model.cpu()
model.train()
# similarity loss definition
loss_fn = torch.nn.CrossEntropyLoss()

# scribble loss definition
loss_fn_scr = torch.nn.CrossEntropyLoss()

# continuity loss definition
loss_hpy = torch.nn.L1Loss(reduction="mean")
loss_hpz = torch.nn.L1Loss(reduction="mean")

HPy_target = torch.zeros(pictures_size - 1, pictures_size, max_nb_class)
HPz_target = torch.zeros(pictures_size, pictures_size - 1, max_nb_class)

if use_cuda:
    HPy_target = HPy_target.cuda()
    HPz_target = HPz_target.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5)
label_colours = np.random.randint(255, size=(max_nb_class,3))


In [10]:
for epoch in range(nepoch):
    for data, _ in dataloader:
        optimizer.zero_grad()
        
        output = model(data)[0]

        output = output.permute(1, 2, 0).contiguous().view(-1, max_nb_class)
        outputHP = output.reshape((pictures_size, pictures_size, max_nb_class))

        HPy = outputHP[1:, :, :] - outputHP[0:-1, :, :]
        HPz = outputHP[:, 1:, :] - outputHP[:, 0:-1, :]
        lhpy = loss_hpy(HPy, HPy_target)
        lhpz = loss_hpz(HPz, HPz_target)

        ignore, target = torch.max(output, 1)

        im_target = target.data.cpu().numpy()
        nLabels = len(np.unique(im_target))
        if visualize:
            im_target_rgb = np.array(
                [label_colours[c % max_nb_class] for c in im_target]
            )

            im_target_rgb = im_target_rgb.reshape((pictures_size, pictures_size, 3)).astype(
                np.uint8
            )
            cv2.imshow("output", im_target_rgb)
            cv2.waitKey(100)

        # loss
        if scribble:
            loss = (
                stepsize_sim * loss_fn(output[inds_sim], target[inds_sim])
                + stepsize_scr
                * loss_fn_scr(output[inds_scr], target_scr[inds_scr])
                + stepsize_con * (lhpy + lhpz)
            )
        else:
            loss = stepsize_sim * loss_fn(output, target) + stepsize_con * (
                lhpy + lhpz
            )

        loss.backward()
        optimizer.step()
        scheduler.step(loss)

    print(
            f"{epoch} / {nepoch} | label num : {nLabels} | loss : {loss.item()}",
        )


0 / 100 | label num : 5 | loss : 4.219082832336426
1 / 100 | label num : 5 | loss : 3.307284116744995
2 / 100 | label num : 5 | loss : 2.782369375228882
3 / 100 | label num : 5 | loss : 1.8548731803894043
4 / 100 | label num : 5 | loss : 2.4779086112976074
5 / 100 | label num : 5 | loss : 1.090612769126892
6 / 100 | label num : 5 | loss : 0.7670782804489136
7 / 100 | label num : 5 | loss : 2.2937545776367188
8 / 100 | label num : 5 | loss : 0.43182921409606934
9 / 100 | label num : 5 | loss : 0.40151649713516235
10 / 100 | label num : 5 | loss : 0.5372830033302307
11 / 100 | label num : 5 | loss : 0.35117754340171814
12 / 100 | label num : 4 | loss : 1.2156084775924683
13 / 100 | label num : 4 | loss : 0.2267618477344513
14 / 100 | label num : 4 | loss : 0.3121795952320099
15 / 100 | label num : 4 | loss : 0.2525736689567566
16 / 100 | label num : 4 | loss : 0.2750709652900696
17 / 100 | label num : 4 | loss : 0.22470223903656006
18 / 100 | label num : 4 | loss : 0.20376259088516235
19

In [5]:
torch.save(model.state_dict(), 'mymodel.save')

In [6]:
model = MyNet(1, max_nb_class, 2)
model.load_state_dict(torch.load('mymodel.save'))
model.eval()

MyNet(
  (conv1): Conv2d(1, 5, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (bn1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): ModuleList(
    (0): Conv2d(5, 5, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  )
  (bn2): ModuleList(
    (0): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Conv2d(5, 5, kernel_size=(1, 1), stride=(1, 1))
  (bn3): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [11]:

tr = transforms.Compose([
     transforms.Grayscale(),
     transforms.ToTensor()]
 )

paths = glob.glob('data/train/*.jpg')

for path in paths:
    img = Image.open(path)
    data = tr(img).unsqueeze(0)

    output = model(data)[0]
    output = output.permute(1, 2, 0).contiguous().view(-1, max_nb_class)
    ignore, target = torch.max(output, 1)
   
    im_target = target.data.cpu().numpy()
    im_target_rgb = np.array([label_colours[c % max_nb_class] for c in im_target])
    im_target_rgb = im_target_rgb.reshape((img.size[1], img.size[0], 3)).astype(np.uint8)

    
    Path("results").mkdir(parents=True, exist_ok=True)
    cv2.imwrite(f"results/{path.rsplit('/',1)[1]}", im_target_rgb)
